In [ ]:
def get_dong(a, b):

    # 이동인구(합) = '*' 값 제거
    mp = a[a['이동인구(합)'] == '*'].index
    a.drop(mp, inplace=True)

    # 생활이동 데이터와 지역코드 데이터 merge
    merge = pd.merge(a, b, how='left', left_on='도착 행정동 코드', right_on='읍면동')
    merge = merge[merge['시도'] == 11000 ] # 서울시도 코드인 '11000'만 추출

    # 이동유형중 도착유형만 사용하기 위해 파생변수 생성
    merge["유형"]=merge["이동유형"].str[1:]

    # 대상연월에서 대상년, 대상월 파생변수 생성
    merge['대상연월'] = pd.to_datetime(merge['대상연월'], format='%Y%m')
    merge['대상년'] = merge['대상연월'].dt.year
    merge['대상월'] = merge['대상연월'].dt.month

    # 필요없는 컬럼 제거 및 데이터 타입 변경
    merge.drop(['출발 행정동 코드', '평균 이동 시간(분)', '시군구', 'full_name', 
                '이동유형', '시도', '읍면동', '대상연월'], axis=1, inplace=True)
    merge =  merge.astype({'이동인구(합)':'float'})

    # 데이터 groupby
    merge = merge.groupby(['대상년', '대상월', '도착시간', '도착 행정동 코드', 'name', 
                           '요일', '유형', '성별', '나이' ], as_index=False).sum()
    
    return merge

In [ ]:
import pandas as pd

code = pd.read_csv('code.csv', encoding='CP949')

def year_month_file(zip_file, hjd_list, code):
    
    df = pd.DataFrame()

    for hjd in hjd_list:
        data = pd.read_csv('./data/'+hjd, encoding='CP949')
        merge = get_dong(data, code)    
        df = pd.concat([df,merge], axis=0)
        
    f_name = re.sub(r'[^0-9]', '', zip_file)
    df.to_csv('생활이동_'+f_name+'.csv', index=False, encoding='ANSI')
        
    print(zip_file , '전처리 데이터(csv) 생성완료')

In [ ]:
def DeleteAllFiles(filePath):
    if os.path.exists(filePath):
        for file in os.scandir(filePath):
            os.remove(file.path)
        return print('Remove All File')
    else:
        return print('Directory Not Found')

In [ ]:
import zipfile
import re
import os 

PATH = "C:/Users/Admin/Downloads/생활이동/" # 압축파일 저장된 폴더

zip_file_list = os.listdir(PATH)

for zip_file in zip_file_list:
    fantasy_zip = zipfile.ZipFile(PATH+zip_file) #년도 월 압축파일
    fantasy_zip.extractall('./data') # data에 압축해제
    
    hjd_list = os.listdir('./data') # data에 압축해제된 년도.월.시간대.파일 리스트

    year_month_file(zip_file, hjd_list, code) # 시간대 파일 합산 데이터 출력
    
    DeleteAllFiles('./data') #압축해제한 데이터 삭제
    print(zip_file, '압축해제한 폴더 삭제 성공')
    
fantasy_zip.close()